# 🚀 Phase 5: Model Deployment

This notebook demonstrates comprehensive model deployment techniques for MLOps, covering model packaging, containerization, and serving.

## Table of Contents
1. [Model Packaging](#1-model-packaging)
2. [Model Containerization](#2-model-containerization)
3. [Model Serving](#3-model-serving)
4. [API Development](#4-api-development)

---

## Prerequisites
Make sure you have the required libraries installed:
```bash
pip install pandas numpy scikit-learn fastapi uvicorn docker joblib
```


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import joblib
import json
import os
from datetime import datetime
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
import docker
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

# Load the best model from Phase 4
try:
    # Try to load the best model
    model_files = [f for f in os.listdir('models/') if f.startswith('best_model_') and f.endswith('.joblib')]
    if model_files:
        model_path = f'models/{model_files[0]}'
        model = joblib.load(model_path)
        print(f"✅ Loaded model: {model_path}")
    else:
        # Create a dummy model for demo
        from sklearn.ensemble import RandomForestClassifier
        model = RandomForestClassifier(random_state=42)
        print("⚠️  Using dummy model for demo")
except:
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(random_state=42)
    print("⚠️  Using dummy model for demo")

print(f"📊 Model type: {type(model).__name__}")


## 1. Model Packaging

**Purpose**: Package model with all dependencies for deployment.


In [ ]:
# 1.1 Step 15: Select Best Model
print("🏆 Step 15: Select Best Model")
print("=" * 50)

# Model metadata
model_metadata = {
    'model_name': type(model).__name__,
    'version': '1.0.0',
    'created_at': datetime.now().isoformat(),
    'model_type': 'classification',
    'target_variable': 'default_risk',
    'features': ['age', 'income', 'education', 'employment_status', 'credit_score', 'loan_amount'],
    'performance_metrics': {
        'accuracy': 0.85,  # Example metrics
        'precision': 0.82,
        'recall': 0.78,
        'f1_score': 0.80,
        'roc_auc': 0.88
    }
}

print(f"Selected Model: {model_metadata['model_name']}")
print(f"Version: {model_metadata['version']}")
print(f"Performance: {model_metadata['performance_metrics']}")

# 1.2 Step 16: Package Model
print("\n📦 Step 16: Package Model")
print("=" * 50)

# Create models directory
os.makedirs('models', exist_ok=True)

# Save model
model_filename = f"models/{model_metadata['model_name'].lower()}_v{model_metadata['version']}.joblib"
joblib.dump(model, model_filename)
print(f"✅ Model saved: {model_filename}")

# Save metadata
metadata_filename = f"models/{model_metadata['model_name'].lower()}_metadata.json"
with open(metadata_filename, 'w') as f:
    json.dump(model_metadata, f, indent=2)
print(f"✅ Metadata saved: {metadata_filename}")

# Create requirements.txt for model
requirements = [
    "pandas>=1.3.0",
    "numpy>=1.21.0",
    "scikit-learn>=1.0.0",
    "joblib>=1.0.0",
    "fastapi>=0.68.0",
    "uvicorn>=0.15.0",
    "pydantic>=1.8.0"
]

with open('models/requirements.txt', 'w') as f:
    f.write('\n'.join(requirements))
print("✅ Requirements saved: models/requirements.txt")

# 1.3 Step 17: Register Model
print("\n📝 Step 17: Register Model")
print("=" * 50)

# Model registry (simplified)
model_registry = {
    'models': {
        model_metadata['model_name']: {
            'version': model_metadata['version'],
            'path': model_filename,
            'metadata_path': metadata_filename,
            'status': 'ready_for_deployment',
            'registered_at': datetime.now().isoformat()
        }
    }
}

# Save registry
registry_filename = 'models/model_registry.json'
with open(registry_filename, 'w') as f:
    json.dump(model_registry, f, indent=2)
print(f"✅ Model registered: {registry_filename}")

print("✅ Model packaging completed!")


## 2. Model Containerization

**Purpose**: Containerize model for scalable deployment.


In [ ]:
# 2.1 Step 18: Containerize Model
print("🐳 Step 18: Containerize Model")
print("=" * 50)

# Create Dockerfile
dockerfile_content = f"""
FROM python:3.9-slim

WORKDIR /app

# Copy requirements and install dependencies
COPY models/requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy model files
COPY models/ ./models/

# Copy API code
COPY api/ ./api/

# Expose port
EXPOSE 8000

# Run the application
CMD ["uvicorn", "api.main:app", "--host", "0.0.0.0", "--port", "8000"]
"""

with open('Dockerfile', 'w') as f:
    f.write(dockerfile_content)
print("✅ Dockerfile created")

# Create docker-compose.yml
docker_compose_content = """
version: '3.8'

services:
  ml-api:
    build: .
    ports:
      - "8000:8000"
    environment:
      - MODEL_PATH=models/randomforestclassifier_v1.0.0.joblib
    volumes:
      - ./models:/app/models
    restart: unless-stopped
"""

with open('docker-compose.yml', 'w') as f:
    f.write(docker_compose_content)
print("✅ docker-compose.yml created")

# Create .dockerignore
dockerignore_content = """
__pycache__
*.pyc
*.pyo
*.pyd
.Python
env
pip-log.txt
pip-delete-this-directory.txt
.tox
.coverage
.coverage.*
.cache
nosetests.xml
coverage.xml
*.cover
*.log
.git
.mypy_cache
.pytest_cache
.hypothesis
"""

with open('.dockerignore', 'w') as f:
    f.write(dockerignore_content)
print("✅ .dockerignore created")

# 2.2 Docker Build Commands
print("\n🔨 Docker Build Commands")
print("-" * 30)

print("To build the Docker image, run:")
print("docker build -t ml-model-api .")
print("\nTo run the container, run:")
print("docker run -p 8000:8000 ml-model-api")
print("\nOr use docker-compose:")
print("docker-compose up --build")

print("✅ Containerization setup completed!")


## 3. Model Serving

**Purpose**: Create API endpoints for model serving.


In [ ]:
# 3.1 Step 19: Deploy Model
print("🚀 Step 19: Deploy Model")
print("=" * 50)

# Create API directory
os.makedirs('api', exist_ok=True)

# 3.2 Step 20: Serve Model via APIs
print("\n🌐 Step 20: Serve Model via APIs")
print("=" * 50)

# Create FastAPI application
api_code = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
import numpy as np
import pandas as pd
from typing import List
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize FastAPI app
app = FastAPI(
    title="ML Model API",
    description="API for Default Risk Prediction",
    version="1.0.0"
)

# Load model
try:
    model = joblib.load("models/randomforestclassifier_v1.0.0.joblib")
    logger.info("Model loaded successfully")
except Exception as e:
    logger.error(f"Error loading model: {e}")
    model = None

# Pydantic models for request/response
class PredictionRequest(BaseModel):
    age: int
    income: float
    education: str
    employment_status: str
    credit_score: int
    loan_amount: float

class PredictionResponse(BaseModel):
    prediction: int
    probability: float
    confidence: str
    model_version: str

# Health check endpoint
@app.get("/health")
async def health_check():
    return {
        "status": "healthy",
        "model_loaded": model is not None,
        "timestamp": pd.Timestamp.now().isoformat()
    }

# Model info endpoint
@app.get("/model_info")
async def model_info():
    if model is None:
        raise HTTPException(status_code=500, detail="Model not loaded")
    
    return {
        "model_type": type(model).__name__,
        "features_count": model.n_features_in_ if hasattr(model, 'n_features_in_') else "unknown",
        "version": "1.0.0"
    }

# Prediction endpoint
@app.post("/predict", response_model=PredictionResponse)
async def predict(request: PredictionRequest):
    try:
        if model is None:
            raise HTTPException(status_code=500, detail="Model not loaded")
        
        # Prepare features
        features = np.array([
            request.age,
            request.income,
            request.credit_score,
            request.loan_amount
        ]).reshape(1, -1)
        
        # Make prediction
        prediction = model.predict(features)[0]
        probability = model.predict_proba(features)[0].max()
        
        # Determine confidence level
        if probability > 0.8:
            confidence = "high"
        elif probability > 0.6:
            confidence = "medium"
        else:
            confidence = "low"
        
        return PredictionResponse(
            prediction=int(prediction),
            probability=float(probability),
            confidence=confidence,
            model_version="1.0.0"
        )
    
    except Exception as e:
        logger.error(f"Prediction error: {str(e)}")
        raise HTTPException(status_code=500, detail="Internal server error")

# Batch prediction endpoint
@app.post("/predict_batch")
async def predict_batch(requests: List[PredictionRequest]):
    try:
        if model is None:
            raise HTTPException(status_code=500, detail="Model not loaded")
        
        # Prepare batch features
        features_list = []
        for req in requests:
            features = [
                req.age,
                req.income,
                req.credit_score,
                req.loan_amount
            ]
            features_list.append(features)
        
        features_array = np.array(features_list)
        
        # Make predictions
        predictions = model.predict(features_array)
        probabilities = model.predict_proba(features_array).max(axis=1)
        
        # Prepare responses
        responses = []
        for i, (pred, prob) in enumerate(zip(predictions, probabilities)):
            confidence = "high" if prob > 0.8 else "medium" if prob > 0.6 else "low"
            responses.append({
                "prediction": int(pred),
                "probability": float(prob),
                "confidence": confidence,
                "model_version": "1.0.0"
            })
        
        return {"predictions": responses}
    
    except Exception as e:
        logger.error(f"Batch prediction error: {str(e)}")
        raise HTTPException(status_code=500, detail="Internal server error")

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

# Save API code
with open('api/main.py', 'w') as f:
    f.write(api_code)
print("✅ API code created: api/main.py")

# 3.3 Step 21: Inference (Real-time Predictions)
print("\n⚡ Step 21: Inference (Real-time Predictions)")
print("=" * 50)

# Test the API locally
print("To test the API locally, run:")
print("uvicorn api.main:app --reload")
print("\nThen visit: http://localhost:8000/docs")
print("\nExample API calls:")
print("""
# Health check
curl http://localhost:8000/health

# Single prediction
curl -X POST "http://localhost:8000/predict" \\
     -H "Content-Type: application/json" \\
     -d '{
       "age": 35,
       "income": 75000,
       "education": "Bachelor",
       "employment_status": "Employed",
       "credit_score": 720,
       "loan_amount": 50000
     }'
""")

print("✅ Model serving setup completed!")
